In [1]:

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)

## A tömeg-rugó-csillapítás rendszer modálanalízise


Egy tömeg-rugó-csillapítás rendszerre alkalmazva a [Modálanalízis](SS-02-Modálanalízis.ipynb) példa eljárását, a feladat bemutatja hogyan lehet a rendszer jellegzetességeit leíró sajátfüggvényeket meghatározni és ábrázolni.
A rendszer mátrix:

$$ A=
\begin{bmatrix}
0 && 1 \\
-\frac{k}{m} && -\frac{c}{m}
\end{bmatrix}$$

és karakterisztikus polinomja (feltéve, hogy az $A$ mátrix irányíthatósági kanonikus alakban prezentált):

$$\lambda^2+\frac{c}{m}\lambda+\frac{k}{m}$$

így a sajátértékek és a hozzájuk tartozó sajátfüggvények:

$$\lambda_{1,2}=-\frac{c}{m}\pm\frac{1}{m}\sqrt{c^2-4km}.$$

A négyzetgyök alatti kifejezés kritikus, mivel ez határozza meg a rendszer viselkedését, hogy lengeni fog-e vagy nem. Ha $c\ge2\sqrt{km}$, akkor a rendszernek csak valós sajátértékei vannak és nem fog lengeni.

Az interaktív példa bemutatja, hogyan változnak a sajátfüggvények a paraméterek változásával.

In [2]:
#Preparatory Cell 
import control
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib import animation

%matplotlib inline

#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
#define the sliders for m, k and c
m = widgets.FloatSlider(
    value=4,
    min=0.1,
    max=10.0,
    step=0.1,
    description='$m$ [kg]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
k = widgets.FloatSlider(
    value=1,
    min=0,
    max=10.0,
    step=0.1,
    description='$k$ [N/m]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
c = widgets.FloatSlider(
    value=4,
    min=0,
    max=10.0,
    step=0.1,
    description='$c$ [Ns/m]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [4]:
#function that make all the computations
def main_callback(m, k, c):
    if c**2-4*k*m >= 0:
        eig1 = -c/m+1/m*numpy.sqrt(c**2-4*k*m)
        eig2 = -c/m-1/m*numpy.sqrt(c**2-4*k*m)
    else:
        eig1 = -c/m+1j*(1/m*numpy.sqrt(-c**2+4*k*m))
        eig2 = -c/m-1j*(1/m*numpy.sqrt(-c**2+4*k*m))
    
    if numpy.real([eig1,eig2])[0] == 0 and numpy.real([eig1,eig2])[1] == 0:
        T = numpy.linspace(0,20,1000)
    else:
        if min(numpy.abs(numpy.real([eig1,eig2]))) != 0:
            T = numpy.linspace(0,7*1/min(numpy.abs(numpy.real([eig1,eig2]))),1000)
        else:
            T = numpy.linspace(0,7*1/max(numpy.abs(numpy.real([eig1,eig2]))),1000)
    if numpy.isreal(eig1):
        if eig1 == eig2:
            mode1 = numpy.exp(eig1*T)
            mode2 = T*numpy.exp(eig2*T)
        else:
            mode1 = numpy.exp(eig1*T)
            mode2 = numpy.exp(eig2*T)
    else:
        mode1 = numpy.exp(eig1.real*T)*numpy.cos(abs(eig1.imag)*T)
        mode2 = numpy.exp(eig2.real*T)*numpy.sin(abs(eig2.imag)*T)
    
    fig = plt.figure(figsize=[16, 5])
    fig.set_label('Modes')
    g1 = fig.add_subplot(121)
    g2 = fig.add_subplot(122)
    
    g1.plot(T,mode1)
    g1.grid()
    g1.set_xlabel('Idő [s]')
    g1.set_ylabel('Első sajátfüggvény')
    
    g2.plot(T,mode2)
    g2.grid()
    g2.set_xlabel('Idő [s]')
    g2.set_ylabel('Második sajátfüggvény')
    
    # print('The eigenvalues are: -%.3f+%.3fj -%.3f-%.3fj' %(abs(eig1.real),abs(eig1.imag),abs(eig2.real),abs(eig2.imag)))
    modesString = 'A sajátértékek $' + str(numpy.around(eig1,decimals=3)) + '$ és $' + str(numpy.around(eig2,decimals=3)) + '$ '
    if numpy.isreal(eig1):
        if eig1 == eig2:
            modesString = modesString + 'a hozzájuk tartozó sajátfüggvények $e^{' + str(numpy.around(eig1,decimals=3))\
                            + ' t}$ és $te^{' + str(numpy.around(eig2,decimals=3)) + ' t}$.'
        else:
            modesString = modesString + 'a hozzájuk tartozó sajátfüggvények $e^{' + str(numpy.around(eig1,decimals=3))\
                            + ' t}$ és $e^{' + str(numpy.around(eig2,decimals=3)) + ' t}$.'
    else:
        modesString = modesString + 'a hozzájuk tartozó sajátfüggvények $e^{' + str(numpy.around(numpy.real(eig1),decimals=3))\
                        + ' t} \cos{(' + str(numpy.around(abs(numpy.imag(eig1)),decimals=3)) + 't)}$ és $e^{'\
                        + str(numpy.around(numpy.real(eig2),decimals=3)) + ' t} \sin{(' + str(numpy.around(abs(numpy.imag(eig2)),decimals=3)) + 't)}$.'
    
    display(Markdown(modesString))

        
out = widgets.interactive_output(main_callback,{'m':m,'k':k,'c':c})
sliders = widgets.HBox([k,m,c])
display(out,sliders)

Output()